# Simulation for Potts model
- All levels of interaction
- Field (constant for each point)
- Preference for low number of colors
- Distance between different colors (0 if equal, 1 of different)

- $N$ vertices
- $P(\sigma) = \frac{1}{N^N} \exp((N-n(\sigma))\gamma$), where $n(\sigma) = \text{# of colors used in } \sigma$

- Consider the interaction on the largest hyperedge on lattice $\Lambda$, where $|\Lambda| = N$
- We assign $\eta_{\Lambda} \in \mathcal{P}(\{1,2,...,q\}^N)$ which is in one-to-one correspondence with $\{1,2,...,N\}$ (a set with $k$ colors corresponds to $k$).
- Each of these corresponds to an energy level in $\{(N-1)\gamma, (N-2)\gamma, ..., 0\}$


- Consider the interaction between adjacent sites (edges) $b$ with size $|b|=2$
- To each edge assign $\eta_{b} \in \mathcal{P}(\{0,1\}^2)$ which is in one-to-one correspondence with the edge being open or closed.
- Each of these corresponds to an energy level in $\{J, -J\}$

- Consider the interaction between adjacent sites and the field, corresponding to $v$ with size $|v|=1$
- To each site assign $\eta_{v} \in \mathcal{P}(\{1,2,...,q\}^N)$ which is in one-to-one correspondence with the color of the site.
- Each of these corresponds to an energy level in $\{(N-1)\alpha, (N-2)\alpha, ...,(N-k)\alpha, ...,0\}$ where $k$ corresponds to the $k$th preferred color of the field.

- Take $ \upsilon_k =  \frac{e^{((N-k)\gamma)} - e^{((N-k-1)\gamma)}}{e^{((N-1)\gamma)}} $

- Joint distribution
- $Q(\eta, \sigma) = \upsilon_{\eta_{\Lambda}} \frac{1}{N^N} \mathbb{1}_{(n(\sigma) \leq \eta_{\Lambda})} $

- Marginals
- $Q(\eta | \sigma) =  \frac { \upsilon_{\eta_{\Lambda}} \frac{1}{N^N} \mathbb{1}_{(n(\sigma) \leq \eta_{\Lambda})} }{P(\sigma)}$
- $Q(\sigma | \eta) = \frac{1}{|\{\sigma:  \eta(\sigma) \leq \eta(\Lambda)\}|} $

In [70]:
# Initial code from https://rajeshrinet.github.io/blog/2014/ising-model/

import numpy as np 
from numpy.random import rand
from scipy.special import binom
from unionfind import UnionFind

# Implement Union Find to find clusters in the configurations
# https://github.com/deehzee/unionfind/blob/master/unionfind.py

def initial_config(N, no_colors=2):   
    ''' Generate a random color/spin configuration for initialization'''
    state = np.random.randint(no_colors, size=(N,N))
    return state


def mc_move(config, eta_prob, eta, N, no_colors, sites):
    '''Monte Carlo move using generalized SW algorithm '''
    # Assign eta to each hyperedge
    eta = assign_etas(config, eta_prob, eta, no_colors)
    # print('assigned eta:', eta)

    # Assign labels to each site
    config = assign_labels(eta, N, no_colors, sites)
    # print('assigned configuration:\n', config)

    return config


def number_of_colors (config):
    '''Number of colors used in the given confuration of labels'''
    return len(np.unique(config))


def avg_sites_per_color (config):
    '''Average number of sites each present color has in the configuration'''
    unq = np.unique(config)
    return config.shape[0]*config.shape[1]/len(unq)


def prob_eta_lambda(gamma, no_colors, k):
    '''Probability of the eta lambda corresponding to a certain number of colors'''
    if k < no_colors:
        return ( np.exp((no_colors-k)*gamma) - np.exp((no_colors-k-1)*gamma) ) / np.exp((no_colors-1)*gamma)
    elif k == no_colors:
        return 1 / np.exp((no_colors-1)*gamma)
    
    
def prob_eta_edge(J, col1, col2):
    '''Probability of the eta edge corresponding to open or not'''
    if col1==col2:
        return 1 - np.exp(-2J)
    else:
        return np.exp(-2J)
    
    
def prob_eta_site(alpha, no_colors, k):
    '''Probability of the eta site corresponding to a maximum color number (lower have higher energy)'''
    if k < no_colors:
        return ( np.exp((no_colors-k)*alpha) - np.exp((no_colors-k-1)*alpha) ) / np.exp((no_colors-1)*alpha)
    elif k == no_colors:
        return 1 / np.exp((no_colors-1)*alpha)

    
def assign_etas(config, eta_prob, eta, no_colors):
    
    '''Assign a number of colors that is at least the current number of colors'''
    eta_lambda = eta[0]
    prob_lambda = eta_prob[0]
    # print('probabilities for no. of colors:', p)
    
    current_k = number_of_colors(config)
    p = prob_lambda[current_k-1:]
    p = p / p.sum()
    # print('normalized:', p)
    eta_lambda = np.random.choice((np.arange(current_k, no_colors+1)), p=p)
    eta[0] = eta_lambda

    '''Assign a closed or open bond to each edge'''
    eta_edges = eta[1]
    prob_edge = eta_prob[1]
    # print('probabilities for edges:', p)
    
    # loop over edges
    for i in range(eta_edges.shape[0]):
        for j in range(eta_edges.shape[1]):
            for e in range(eta_edges.shape[2]):
                if eta_edges[i,j,e] == -1: continue
                if e==0: current_b = (0 if config[i,j]==config[i+1,j] else 1)          
                elif e==1: current_b = (0 if config[i,j]==config[i,j+1] else 1) 
                    
                p = prob_edge[current_b:]
                p = p / p.sum()
                # print('normalized:', p)
                eta_edge = np.random.choice((np.arange(current_b, 1+1)), p=p) # 0 for present bonds, 1 for not
                eta_edges[i,j,e] = eta_edge
    eta[1] = eta_edges
    
    '''Assign a color that has at most the current energy for the color'''
    eta_sites = eta[2]
    prob_site = eta_prob[2]
    
    # loop over sites
    for i in range(eta_sites.shape[0]):
        for j in range(eta_sites.shape[1]):
            current_s = config[i,j]
            p = prob_site[current_s:]
            p = p / p.sum()
            # print('normalized:', p)
            eta_site = np.random.choice((np.arange(current_s, no_colors+1)), p=p)
            eta_sites[i,j] = eta_site
    eta[2] = eta_sites
            
    return eta


def site2str(tup):
    return str(tup[0]) + ',' + str(tup[1])

def str2site(strr):
    t = strr.split(',')
    return [int(x) for x in t]


def sample_config(eta, N, no_colors, sites):
        
    '''Generate clusters from the assigned bonds (eta_edge)'''

    uf = UnionFind(sites)
    
    
    
    uf.find('e')
    uf.components()

    '''For each cluster, find the site with strongest constraint (smallest eta_site)'''
    '''Assign that eta_site to the entire cluster'''
    
    '''Randomly sample a color for each cluster'''

    
def assign_labels(eta, N, no_colors, sites):
    '''Assign a color configuration chosen uniformly from the configurations compatible with eta'''
    # Brute force version
    #     max_colors = eta[0] # eta_lambda
    #     config = sample_config(eta, N, no_colors, sites)
    #     while number_of_colors(config) > max_colors:
    #         config = sample_config(eta, N, no_colors, sites)
    #     return config
    
    return config

In [ ]:
from unionfind import UnionFind

# Implement The Hoshen-Kopelman Algorithm using Union Find to find clusters in the configurations
# https://www.ocf.berkeley.edu/~fricke/projects/hoshenkopelman/hoshenkopelman.html
# https://github.com/deehzee/unionfind/blob/master/unionfind.py

In [28]:
# Small experiment

N, q = 3, 4
gamma = 2.5
J = 1
alpha = 2

'''Probabilities ordered from highest (high energy) to lowest'''
lambda_prob = np.zeros(q)
for j in range(q):
    lambda_prob[j] = lambda_prob(gamma,q,j+1)
print('lambda probabilities:', lambda_prob)

edge_prob = np.zeros(2)
edge_prob[0] = prob_eta_edge(J, 'same_col', 'same_col')
edge_prob[1] = prob_eta_edge(J, 'same_col', 'dif_col')
print('edge probabilities:', edge_prob)

site_prob = np.zeros(q)
for j in range(q):
    site_prob[j] = site_prob(alpha,q,j+1)
print('site probabilities:', site_prob)

eta_prob = (lambda_prob, edge_prob, site_prob)

'''Current states of eta'''
eta_lambda = 0
eta_edges = np.zeros((N,N,2))
# Special edge cases (no neighbors at the border)
eta_edges[:,N-1,1] = -1
eta_edges[N-1,:,0] = -1
eta_sites = np.zeros((N,N))
eta = (eta_lambda, eta_edges, eta_sites)


'''List of sites (tuples) for Union-Find'''
sites = []
for i in range(N):
    for j in range(N):
        sites.append(str(i)+','+str(j))

config = initial_config(N,q)
print('initial config:\n', config)
config = mc_move(config, eta_prob, eta, N, q, sites)

SyntaxError: EOL while scanning string literal (<ipython-input-28-f8f7c13ecffa>, line 26)

In [75]:
from unionfind import UnionFind

N = 5

class Site:
  def __init__(self, x, y):
    self.x = x
    self.y = y

# sites = np.zeros((N,N)).tolist()
sites = []
for i in range(N):
    for j in range(N):
        sites.append(str(i)+','+str(j))
#         sites[i][j] = (i,j) #Site(i,j))
        
uf = UnionFind(sites)
uf

<UnionFind:
	elts=['0,0', '0,1', '0,2', '0,3', '0,4', '1,0', '1,1', '1,2', '1,3', '1,4', '2,0', '2,1', '2,2', '2,3', '2,4', '3,0', '3,1', '3,2', '3,3', '3,4', '4,0', '4,1', '4,2', '4,3', '4,4'],
	siz=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
	par=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
n_elts=25,n_comps=25>

In [76]:
uf.union('1,1', '0,1')
uf.union('2,1', '1,1')
uf.components()
# uf.component('0,1').pop().split(',')


# uf.union(Site(1,1), Site(0,1))
# uf.union(Site(2,1), Site(1,1))
# uf
# uf.find('1,1')
# uf.component


[{'0,0'},
 {'0,2'},
 {'0,3'},
 {'0,4'},
 {'1,0'},
 {'0,1', '1,1', '2,1'},
 {'1,2'},
 {'1,3'},
 {'1,4'},
 {'2,0'},
 {'2,2'},
 {'2,3'},
 {'2,4'},
 {'3,0'},
 {'3,1'},
 {'3,2'},
 {'3,3'},
 {'3,4'},
 {'4,0'},
 {'4,1'},
 {'4,2'},
 {'4,3'},
 {'4,4'}]